# 自定义训练/评估循环(自行实现 fit)

`fit` 在易用性和灵活性之间取得了非常好的平衡,但是这并不意味着 fit 可以适用于任何训练过程.即使 fit 配合自定义指标 自定义损失 自定义回调等,也无法满足全部的训练场景.

`fit` 的工作流只适用于监督学习,监督学习要求数据已经标注,之后通过训练得出模型.然而监督学习只是机器学习的一个大类,还有无监督学习,半监督学习,生成学习,强化学习等等.

如果 fit 无法满足需要,此时就需要编写自己的训练逻辑了.第 2 3 章我们见过一些简单的低层次训练逻辑.一个典型的训练循环如下

- 计算正向传播(人话就是计算模型输出),获取当前批次的损失值.
- 检索与模型权重有关的损失梯度
- 更新模型权重,以降低当前批次的损失值.

以上其实就是 fit 做的全部事情,这一节会从头重新实现 fit.


## 训练/推理

在 2 3 章的例子中

- 正向传播: `predictions = model(inputs)`
- 检索与权重相关梯度: `gradients = tape.gradient(loss, model.weights`

编写自定义 fit 这里有两个问题

一些 keras 层在训练和预测之间的行为并不相同.在前向传递调用 keras 模型时,一定要将 `training` 设置为 true.

- 例如 dropout 层在其 `call` 方法中有意训练的 bool 参数,调用 `dropout(inputs, training=True)` 将会忽略一些激活单元,但是调用 `dropout(inputs, training=Flase)` 则不会忽略.
- 这个参数在 Sequential 或 Functional 生成的模型中也存在.

检索模型的权重梯度时应该使用 `tape.gradients(loss, model.trainable_weights)` 而不是 `tape.gradients(loss, model.weights)`.实际上层和模型有两种类型的权重

- 可训练权重: 可以通过反向传播更新,以最小化模型损失.例如米基础的 kernel 和 bias.
- 不可训练权重: 前向传递过程中,由层自行维护更新的权重.例如在层增加一个批次计数器.这个信息就会存放在不可训练权重中,每个批次训练时层将计数器 +1.

keras 内置的层中,唯一具有不可训练权重的层是 BatchNormalization ,我们将在第 9 章见到.BatchNormalization 使用不可训练权重跟踪它数据的平均值和标准差,~~进行及时的特征标准化(归一化/规范化).~~(这一句翻译感觉有问题,等到第9章再说).

In [ ]:
import tensorflow as tf

In [ ]:
def train_step(inputs, targets):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)
        loss = loss_fn(targets, predictions)
    gradients = tape.gradients(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(model.trainable_weights, gradients))